In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import copy
from math import *
from functools import reduce
from PIL import Image as Img
from PIL import ImageTk
from PIL import ImageEnhance
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from sklearn.cluster import KMeans

In [ ]:
# reading in dicom files
import pydicom as dicom

In [ ]:
# skimage image processing packages
import skimage
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
from skimage.transform import resize

In [ ]:
# scipy linear algebra functions 
from scipy.linalg import norm
import scipy.ndimage

In [ ]:
# ipywidgets for some interactive plots
from ipywidgets.widgets import * 
import ipywidgets as widgets

In [ ]:
# plotly 3D interactive graphs 
import plotly
from plotly.graph_objs import *
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
import chart_studio.plotly as py

In [ ]:
# set path and load files 
#path = r'C:\Users\adamj\OneDrive\Desktop\DICOM Python\Dicom Files'
#output_path = r'C:\Users\adamj\OneDrive\Desktop\DICOM Python\Numpy Data'
path = '/Users/adamj/OneDrive/Desktop/DICOM Python/Dicom Files/'
output_path = working_path = '/Users/adamj/OneDrive/Desktop/DICOM Python/Numpy Data/'
g = glob(path + '/*.dcm')

In [ ]:
# checking we're in the correct path
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print ('\n'.join(g[:5]))

In [ ]:
# https://hengloose.medium.com/a-comprehensive-starter-guide-to-visualizing-and-analyzing-dicom-images-in-python-7a8430fcb7ed
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
# Load all DICOM images from a folder into a list for manipulation

def load_scan(path):
    ct_images = os.listdir(path)
    slices = [dicom.dcmread(path + '/' + s) for s in ct_images]
    slices = [s for s in slices if 'SliceLocation' in s]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    
    return slices

In [ ]:
# https://hengloose.medium.com/a-comprehensive-starter-guide-to-visualizing-and-analyzing-dicom-images-in-python-7a8430fcb7ed
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
# Converts all raw voxel values into Houndsfeld Units. 
# HU's are useful because it is standardized across all CT scans regardless 
# of the absolute number of photons the scanner detector captured.

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfeld units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
   
    return np.array(image, dtype=np.int16)

In [ ]:
patient = load_scan(path)
patient_pixels = get_pixels_hu(patient)

In [ ]:
height, width, depth = patient_pixels.shape
print(f'The image object has the following dimensions:\nheight={height}\nwidth={width}\ndepth={depth}.')

In [ ]:
id = 0
np.save(output_path + "fullimages_%d.npy" % (id), patient_pixels)

In [ ]:
def hide_axes(): # gets rid of x and y axes on plots if desired
    ax = plt.gca()
    plt.axis('off')
    #ax.get_xaxis().set_visible(False)
    #ax.get_yaxis().set_visible(False)

In [ ]:
# We want to check whether the Houndsfeld Units are properly scaled and represented
file_used=output_path+"fullimages_%d.npy" % id
imgs_to_process = np.load(file_used).astype(np.float64)

plt.hist(imgs_to_process.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Houndsfeld Units of various substances in the human body, for reference.
HU_table = Img.open("Houndsfeld Units Table.jpg")
plt.figure(figsize=(8,8))
plt.imshow(HU_table)
hide_axes()

In [ ]:
# Critical Analysis of Histogram
# High volume of air
# Fat, Water, Blood, Muscle and Liver may be present, but difficult to determine the quantity of each
# Small bits of bone are present (mainly cancellous bone)

In [ ]:
# We will skip every 4 slices to produce an image stack to see what we are looking at.
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
id = 0
imgs_to_process = np.load(output_path+'fullimages_{}.npy'.format(id))

def sample_stack(stack, rows=7, cols=7, start_with=1, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs_to_process)

In [ ]:
print("Slice Thickness: %f" % patient[0].SliceThickness)
print("Pixel Spacing (row, col): (%f, %f) " % (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))

In [ ]:
"""This means each slice is 1.5 mm thick and each voxel represents approximately 0.7 mm. A CT slice is typically
reconstructed at 512x512 voxels, which means each slice represents approximately 379 mm of data in length and width."""

In [ ]:
# Displaying the image in 3D isometric form.
"""Using the metadata from the DICOM Files we can determine the size of each voxel as the slice thickness. To be able
to display a 3D isometric image, we need to resample each slice into 1x1x1 mm pixels and slices"""

In [ ]:
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/

id = 0
imgs_to_process = np.load(output_path+'fullimages_{}.npy'.format(id))
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]]))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print("Shape before resampling\t", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1,1,1])
print("Shape after resampling\t", imgs_after_resamp.shape)

In [ ]:
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/

def make_mesh(image, threshold=-300, step_size=1):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces

In [ ]:
def plot_3d(verts, faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=0.1)
    face_color = [0.5, 0.5, 1] # sets the colour of the image (RGB Format)
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_facecolor((0.5, 0.5, 0.5))
    plt.show()
    
vert, face = make_mesh(imgs_after_resamp, 350)
plot_3d(vert, face)

In [ ]:
def plotly_3d(verts, faces): # interactive 3D visualisation
    x,y,z = zip(*verts) 
    
    print("Drawing")
    
    # Make the colormap single color since the axes are positional not intensity. 
    #    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    colormap=['rgb(227,219,201)','rgb(227,219,201)']  #rgb(227,219,201) rgb(236, 236, 212)
    
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor='rgb(64, 64, 64)',
                        title="Interactive Visualization")
    iplot(fig)
    
vert, face = make_mesh(imgs_after_resamp, 350, 2)
plotly_3d(vert, face)

In [ ]:
# Segmentation - Applying mask
# code from: https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
# The below code will:

# Standardize the pixel value by subtracting the mean and dividing by the standard deviation
# Identify the proper threshold by creating 2 KMeans clusters comparing centered on soft tissue/bone vs lung/air.
# Using Erosion) and Dilation) which has the net effect of removing tiny features like pulmonary vessels or noise
# Identify each distinct region as separate image labels (think the magic wand in Photoshop)
# Using bounding boxes for each image label to identify which ones represent lung and which ones represent "every thing else"
# Create the masks for lung fields.
# Apply mask onto the original image to erase voxels outside of the lung fields.

In [ ]:
def make_mask(img, display=False):
    row_size = img.shape[0]
    col_size = img.shape[1]
    
    # Standardise the pixel values
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    #
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    #
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    #
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation

    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
    return mask*img

In [ ]:
img = imgs_after_resamp[140]
make_mask(img, display=True)

In [ ]:
masked_lung = []

for img in imgs_after_resamp:
    masked_lung.append(make_mask(img))

sample_stack(masked_lung, show_every=5)

In [ ]:
mask3D = []
for img in imgs_after_resamp:
    mask3D.append(make_mask(img))

In [ ]:
len(mask3D)

In [ ]:
vert, face = make_mesh(mask3D, 350, 2)
plotly_3d(vert, face)